# MeanShift ... The Object Tracking Algorithm

In [1]:
import cv2
import numpy as np

# initialize the webcam
cap= cv2.VideoCapture(0) 

# code that takes the first frame of the video
ret, frame = cap.read()
print type(frame)

# code that sets up the default loaction of the window
r, h, c, w = 240, 100, 400, 160
track_window = (c, r, w, h)

# code that crops the region of interest for tracking
roi = frame[r:r+h, c:c+w]

# code that converts cropped window to HSV color space
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# creates a mask between the HSV upper and lower bounds
lower_purple = np.array([125, 0, 0])
upper_purple = np.array([175, 255, 255])
mask = cv2.inRange(hsv_roi, lower_purple, upper_purple)

# code that obtains the color Histogram of the ROI
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [100], [0, 180])

# code that normalize values to lie between the range 0, 255
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)


# setups the termination criternia of the meanShift algo
# the code below says that we will stop calculating the centroid shift after ten iterations
# or if the centroid has moved by "1" pixel
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while True:
    # code that reads the webcam frame
    ret, frame = cap.read()
    
    frame = cv2.flip(frame, 1)
    
    if ret == True:
        
        # converts to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # calculates histogram back projection
        # Each pixel value is it's probability. N.b : This is what MeanShift uses to track the object
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        
        # code that applies meanshift to get the new_loaction of the object
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        
        # code that draws this on image
        x, y, w, h = track_window
        img2 = cv2.rectangle(frame, (x, y), (x+w, y+h), 255, 2)
        
        cv2.imshow('MeanShift Tracking', img2)
        
        if cv2.waitKey(1) == 13:
            break
    else:
        break
            

cv2.destroyAllWindows()
cap.release()

<type 'numpy.ndarray'>


# What are cv2.calcHist and cv2.calcBackProject

cv2.calcHist is a function that calculates the color histograms of an array of images

cv2.calcBackProject calculates histogram back projection

i.e it takes histogram generated by Calcist and projects it back onto an image. The result is the probability that each pixel belongs to the image that was originally generated by the histogram.